In [1]:
 # Создаем spark сессию
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .config('spark.driver.memory', '16G')
    .config('spark.sql.analyzer.failAmbiguousSelfJoin', 'False')
    .master("local[*]")
    .getOrCreate()
)
spark.sparkContext.setCheckpointDir("/temp/checkpoints")

In [2]:
from surprise.dataset import Dataset
data = Dataset.load_builtin(name="ml-1m")

In [3]:
DATA_DIR = 'c:/Users/adwiz/.surprise_data/ml-1m/ml-1m/'
!sed 's/::/,/g' {DATA_DIR}/ratings.dat > {DATA_DIR}/ratings.csv
!head {DATA_DIR}/ratings.csv

1,1193,5,978300760
1,661,3,978302109
1,914,3,978301968
1,3408,4,978300275
1,2355,5,978824291
1,1197,3,978302268
1,1287,5,978302039
1,2804,5,978300719
1,594,4,978302268
1,919,4,978301368


In [4]:
import os
import pyspark.sql.functions as sql_func

# читаем файл с оценками (26 млн)
ratings = (
    spark
    .read
    .csv(
        os.path.join(DATA_DIR, "ratings.csv"),
        header = False,
        inferSchema = True
    )
    .toDF("userId", "movieId", "rating", "timestamp")
    .select("userId", "movieId", "rating")
)

In [5]:
ratings.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|   1193|     5|
|     1|    661|     3|
|     1|    914|     3|
|     1|   3408|     4|
|     1|   2355|     5|
|     1|   1197|     3|
|     1|   1287|     5|
|     1|   2804|     5|
|     1|    594|     4|
|     1|    919|     4|
|     1|    595|     5|
|     1|    938|     4|
|     1|   2398|     4|
|     1|   2918|     4|
|     1|   1035|     5|
|     1|   2791|     4|
|     1|   2687|     3|
|     1|   2018|     4|
|     1|   3105|     5|
|     1|   2797|     4|
+------+-------+------+
only showing top 20 rows



In [6]:
# разбиваем на tarain test
train, test = ratings.randomSplit([.8, .2], seed=42)
train.cache()
test.cache()

DataFrame[userId: int, movieId: int, rating: int]

In [7]:
# %%time
# from pyspark.ml.recommendation import ALS

# # обучаем модель с базовыми настройками
# als = ALS(
#     userCol="userId",
#     itemCol="movieId",
#     ratingCol="rating",
#     # если в тестовой выборке будет встречаться неизвестный фильм
#     # или неизвестный пользователь, они будут игнорироваься
#     coldStartStrategy="drop",
#     seed=42
# ).fit(train)

In [8]:
%%time
from pyspark.ml.recommendation import ALS

# обучаем модель с настройками аналогичными surprise
als = ALS(
    rank=100,
    maxIter=40,
    checkpointInterval=5,
    userCol="userId",
    itemCol="movieId",
    ratingCol="rating",
    # если в тестовой выборке будет встречаться неизвестный фильм
    # или неизвестный пользователь, они будут игнорироваься
    coldStartStrategy="drop",
    seed=42
).fit(train)

Wall time: 2min 6s


In [9]:
%%time
from pyspark.ml.evaluation import RegressionEvaluator

# Оцениваем RMSE
predictions = als.transform(test)
rmse = evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="rating",
    predictionCol="prediction"
).evaluate(predictions)
print("RMSE на тестовой выборке: ", rmse)

RMSE на тестовой выборке:  0.8585006953935764
Wall time: 22.2 s
